In [13]:
import pandas as pd
import mysql.connector
from mysql.connector import errorcode

In [14]:
data_df = pd.read_csv('/Users/ksenia/Analytics/Instacart/products.csv')

In [15]:
data_df[:5]

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [16]:
data_df.dtypes

product_id        int64
product_name     object
aisle_id          int64
department_id     int64
dtype: object

In [17]:
def get_conn():
    return mysql.connector.connect(host= "localhost",
                  user="root",
                  passwd="password",
                  db="instacart")

In [18]:
# Create table Products in MySQL
TABLES = {}
TABLES['products'] = (
    "CREATE TABLE `products` ("
    "  `product_id` int NOT NULL,"
    "  `product_name` varchar(200),"
    "  `aisle_id` int,"
    "  `department_id` int,"
    "  PRIMARY KEY (`product_id`), UNIQUE KEY `id` (`product_id`)"
    ") ENGINE=InnoDB")

conn = get_conn()
cursor = conn.cursor()
for name, ddl in TABLES.iteritems():
    try:
        print("Creating table {}: ", name)
        cursor.execute(ddl)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")
conn.commit()
cursor.close()
conn.close()

('Creating table {}: ', 'products')
OK


In [21]:
#List of headers for inserting data into database
headers = 'product_id, product_name, aisle_id, department_id'
headers = headers.split(", ")
print(headers)

['product_id', 'product_name', 'aisle_id', 'department_id']


In [31]:
#Function to insert each line with into database under corresponding headers 
def create_insert_sql(line, headers):
    """
    Create a SQL statement for inserting row 
    """
    values = []
    for header in headers:
        val = line[header]
        
        if repr(val) == 'nan':
            values.append('null')
        else:
            values.append(repr(val))

    headerpart = ', '.join(headers)    
    valuespart = ', '.join(values)

    sql = 'INSERT INTO instacart.products (' + headerpart + ')' + ' VALUES' + '(' + valuespart + ')'
    return sql


#Test funtion
line = data_df.iloc[2,:]
print(create_insert_sql(line, headers))

INSERT INTO instacart.products (product_id, product_name, aisle_id, department_id) VALUES(3, 'Robust Golden Unsweetened Oolong Tea', 94, 7)


In [33]:
#Write product data into database

conn = get_conn()
cursor = conn.cursor()

try:
    for row in range(0,data_df.shape[0]):
        if row % 1000 == 0: 
            print(row)
            conn.commit()
        line = data_df.iloc[row,0:]
        sql = create_insert_sql(line, headers)
        try:
            cursor.execute(sql)
        except Exception as e:
            print(sql)
            raise e
    print("all done")

    
finally:
    cursor.close()
    conn.close()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
all done
